In [10]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [11]:
VERSION = "2"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [12]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [13]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
}

In [14]:
LOSS = {
    "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy(),
    "BinaryFocalCrossentropy": tf.keras.losses.BinaryFocalCrossentropy(
        alpha=0.25, gamma=2.0
    ),
    "SigmoidFocalCrossEntropy": tfa.losses.SigmoidFocalCrossEntropy(
        alpha=0.25, gamma=2.0
    ),
    "FocalLoss": models.FocalLoss(alpha=0.25, gamma=2.0),
}

In [15]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [16]:
BATCH_SIZE = 64

In [17]:
SUBVERSION = 0
for LOSS_NAME, _ in LOSS.items():
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS[LOSS_NAME],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-01 17:05:27.157956: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.2820 - accuracy: 0.8738 - precision: 0.8962 - recall: 0.9378 - auc: 0.9355

2025-03-01 17:06:16.526180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 61s 213ms/step - loss: 0.2820 - accuracy: 0.8738 - precision: 0.8962 - recall: 0.9378 - auc: 0.9355 - val_loss: 0.2185 - val_accuracy: 0.9134 - val_precision: 0.9308 - val_recall: 0.9390 - val_auc: 0.9695
Epoch 2/10
280/280 [==============================] - 58s 206ms/step - loss: 0.2077 - accuracy: 0.9133 - precision: 0.9381 - recall: 0.9450 - auc: 0.9662 - val_loss: 0.1924 - val_accuracy: 0.9216 - val_precision: 0.9386 - val_recall: 0.9432 - val_auc: 0.9760
Epoch 3/10
280/280 [==============================] - 57s 203ms/step - loss: 0.1897 - accuracy: 0.9234 - precision: 0.9460 - recall: 0.9506 - auc: 0.9719 - val_loss: 0.1874 - val_accuracy: 0.9260 - val_precision: 0.9446 - val_recall: 0.9436 - val_auc: 0.9761
Epoch 4/10
280/280 [==============================] - 57s 204ms/step - loss: 0.1824 - accuracy: 0.9249 - precision: 0.9483 - recall: 0.9502 - auc: 0.9740 - val_loss: 0.1830 - val_accuracy: 0.9263 - val_precision: 0.9390 - val_recall: 

...

280/280 [==============================] - 61s 213ms/step - loss: 0.0183 - accuracy: 0.8978 - precision: 0.9281 - recall: 0.9310 - auc: 0.9574 - val_loss: 0.0132 - val_accuracy: 0.9176 - val_precision: 0.9493 - val_recall: 0.9249 - val_auc: 0.9733
Epoch 2/10
280/280 [==============================] - 59s 209ms/step - loss: 0.0124 - accuracy: 0.9199 - precision: 0.9440 - recall: 0.9478 - auc: 0.9723 - val_loss: 0.0131 - val_accuracy: 0.9195 - val_precision: 0.9618 - val_recall: 0.9147 - val_auc: 0.9752
Epoch 3/10
280/280 [==============================] - 60s 213ms/step - loss: 0.0115 - accuracy: 0.9243 - precision: 0.9481 - recall: 0.9494 - auc: 0.9760 - val_loss: 0.0122 - val_accuracy: 0.9265 - val_precision: 0.9497 - val_recall: 0.9386 - val_auc: 0.9771
Epoch 4/10
280/280 [==============================] - 57s 204ms/step - loss: 0.0111 - accuracy: 0.9280 - precision: 0.9519 - recall: 0.9506 - auc: 0.9779 - val_loss: 0.0116 - val_accuracy: 0.9300 - val_precision: 0.9418 - val_recall: 

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV2/v2/v2.3/assets


67/67 [==============================] - 12s 175ms/step - loss: 0.0130 - accuracy: 0.9221 - precision: 0.9134 - recall: 0.9746 - auc: 0.9813


2025-03-01 17:46:35.443240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 11s 164ms/step



